# Extract Sv from ANMPA data
This notebook will be used to calculate the mean backscatter value for the 2013 and 2014 data from the surveys in the ANMPA area.

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from mpl_toolkits.basemap import Basemap

## Evaluation zone
The area that will be assesed for mean backscatter is determine by the stations used in all the survey years. 

In [43]:
min_lat = 69.39130000000
max_lat = 70.44650000000

min_long = -125.85600000000
max_long = -123.19400000000


## Evaluation dates
The boats were at the stations location only over certain days. To cut down on processing time we will only look at the time the boats were at the stations

2013: 08.02 to 08.08

2014: 08.17, 08.22, 08.23, 08.25, 08.26

2017:08.15, 08.16, 08.18, 08.23 to 08.27

2018: 08.08, 08.09, 08.30, 08.31, 09.01, 09.04, 09.08


**Q: Should it be ALL the date BETWEEN the start and end of the stations each year or JUST the days it is AT a station? For now I will do just the specific days to reduce processing time.**

In [51]:
values_2013 = pd.DataFrame(columns={'Sv','Lats','Longs','Depth'})
values_2014 = pd.DataFrame(columns={'Sv','Lats','Longs','Depth'})

In [52]:
days_2012 = {'6 au 21 aout','22 aout au 3 sept'}
fname = '../data/AmundsenGulf/2012/%s (biomasse).csv' %date
values_2012 = pd.read_csv(fname, header=0, usecols=[6,10,32,33])
data12 = values_2012.values  


#a)2013
days_2013 = {'02','03','04','05','06','07','08'}
for date in days_2013:
    fname = '../data/AmundsenGulf/Biomass_2013/%s-08-13 (biomasse).csv' %date
    # Columns are SV-mean, depth, Lat, long.
    values_file = pd.read_csv(fname, header=0, usecols=[6,10,32,33])
    values_2013.append(values_file)
    
# Change to a numpy array
print(values_2013)
data13 = values_2013.values  



#b)2014
days_2014 = {'17-08','22-08','23-08','24-08','25-08','26-08','27-08'}
for date in days_2014:
    fname = '../data/AmundsenGulf/Biomass_2014/%s-14 (biomasse).csv' %date
    # Columns are SV-mean, depth, Lat, long.
    values_file = pd.read_csv(fname, header=0, usecols=[6,10,32,33])
    values_2014.append(values_file)
    fname2 = '../data/AmundsenGulf/Biomass_2014/%s-14 (biomasse) 2.csv' %date
    # Columns are SV-mean, depth, Lat, long.
    values_file = pd.read_csv(fname2, header=0, usecols=[6,10,32,33])
    values_2014.append(values_file)
    
# Change to a numpy array
print(values_2014)
data14 = values_2014.values  

Empty DataFrame
Columns: [Sv, Lats, Longs, Depth]
Index: []
Empty DataFrame
Columns: [Sv, Lats, Longs, Depth]
Index: []


## Select values

### A) 0-100 m from the surface

In [46]:
#a)2013
data13_under100 = data13[data13[:,1]<100,:]
print(np.shape(data13_under100))


#b)2014
data14_under100 = data14[data14[:,1]<100,:]
print(np.shape(data14_under100))

IndexError: index 1 is out of bounds for axis 1 with size 0

### B) Within lat and long station box


In [ ]:
#a)2013
index13_within = np.where((data13_under100[:,2]>min_lat)*(data13_under100[:,2]<max_lat)*(data13_under100[:,3]>min_long)*(data13_under100[:,3]<max_long))
data13_within=data13_under100[index13_within[0],:]
print(np.shape(data13_within))

#b)2014
index14_within = np.where((data14_under100[:,2]>min_lat)*(data14_under100[:,2]<max_lat)*(data14_under100[:,3]>min_long)*(data14_under100[:,3]<max_long))
data14_within=data14_under100[index14_within[0],:]
print(np.shape(data14_within))

## Check values that are left, remove outliers.
Scatter plot SV in 2013 and 2014
**Q: Why are there -1000dB values?**

In [ ]:
plt.subplot(211)
plt.plot(data13_within[:,0], '.')
plt.title('2013 Volume backscatter')


plt.subplot(212)
plt.plot(data14_within[:,0], '.')
plt.title('2014 Volume backscatter')

In [ ]:
#Remove the -1000 dB values.
data13_trimmed = data13_within[data13_within[:,0]>-900,:]
data14_trimmed = data14_within[data14_within[:,0]>-900,:]

In [ ]:
plt.figure(figsize=(10,10))
ax1 = plt.subplot(211)
ax1.plot(data13_trimmed[:,0], '.')
ax1.set_title('2013 Volume backscatter - Trimmed')
ax1.set_ylabel('Sv [dB]')

ax2 = plt.subplot(212)
ax2.plot(data14_trimmed[:,0], '.')
ax2.set_title('2014 Volume backscatter - Trimmed')
ax2.set_ylabel('Sv [dB]')

## Calculate linear mean of Sv
Volume backscattering strength to volume backscattering coefficient

$S_{v} = 10 log(s_{v})$

Therefore,

$s_{v} = 10^\frac{S_{v}}{10}$


In [ ]:
#a)2013
S_v13 = data13_trimmed[:,0]
sv13 = 10**(S_v13/10)
mean_sv13 = np.mean(sv13)
mean_S_v13 = 10*np.log10(mean_sv13)


#b)2014
S_v14 = data14_trimmed[:,0]
sv14 = 10**(S_v14/10)
mean_sv14 = np.mean(sv14)
mean_S_v14 = 10*np.log10(mean_sv14)

## Calculate std in log space

In [ ]:
#a) 2013
std_sv13 = np.std(sv13)
stdm_Sv13 = np.log10(np.abs(mean_sv13 - std_sv13))
stdp_Sv13 = np.log10(np.abs(mean_sv13 + std_sv13))

#b) 2014
std_sv14 = np.std(sv14)
stdm_Sv14 = np.log10(np.abs(mean_sv14 - std_sv14))
stdp_Sv14 = np.log10(np.abs(mean_sv14 + std_sv14))



In [ ]:
print('2013 mean SV = %f - %f / + %f \n2014 mean SV = %f - %f / + %f' %(mean_S_v13, stdm_Sv13, stdp_Sv13, mean_S_v14, stdm_Sv14, stdp_Sv14))

## Map location of datapoints
**Q: Better res map data point available?**

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()

ax.set_ylim([min_lat-1.05, max_lat+1.5])
ax.set_xlim([min_long-1.005, max_long+1.005])
plt.plot(data13_trimmed[:,3],data13_trimmed[:,2] ,color='blue', linewidth=0.5, marker='.',
         transform=ccrs.Geodetic(),)
plt.plot(data14_trimmed[:,3],data14_trimmed[:,2] ,color='red', linewidth=0.5, marker='.',
         transform=ccrs.Geodetic(),)

In [ ]:
plt.figure(figsize=(10,10))
m = Basemap(projection='npstere',boundinglat=min_lat,lon_0=5,resolution='l')
m.drawcoastlines()
m.fillcontinents(color='coral',lake_color='aqua')
m.drawparallels(np.arange(-80.,81.,20.))
m.drawmeridians(np.arange(-180.,181.,20.))
m.drawmapboundary(fill_color='aqua')
# draw tissot's indicatrix to show distortion.
ax = plt.gca()

plt.plot(data13_trimmed[:,3],data13_trimmed[:,2] ,color='blue', linewidth=0.5, marker='.')

plt.plot(data14_trimmed[:,3],data14_trimmed[:,2] ,color='red', linewidth=0.5, marker='.')